In [1]:
import mysql.connector
import pandas as pd 
import plotly.express as px

In [16]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="kr1997",
  database="bourses2",
  auth_plugin="mysql_native_password"
    #port=
)

In [7]:
del mydb

In [17]:
mycursor = mydb.cursor()
mycursor.execute("show FULL tables;")
myresult = mycursor.fetchall()
print(myresult)

[('academie', 'BASE TABLE'), ('bourse', 'BASE TABLE'), ('groupe_candidats', 'BASE TABLE'), ('institut', 'BASE TABLE'), ('region', 'BASE TABLE'), ('secteur', 'BASE TABLE'), ('type_bourse', 'BASE TABLE'), ('type_excellence', 'BASE TABLE')]


In [18]:
dx = pd.read_sql_query("select *  from secteur;", mydb);
print(dx)

   code_secteur_cnu          secteur_disciplinaire
0                 1  Droit et sciences économiques
1                 2   Lettres et sciences humaines
2                 3                       Sciences
3                 4                          Santé
4                 5                          STAPS


# Upload pre-processed CSV file

In [28]:
df = pd.read_csv('bourses_vf.csv',sep=';')  

In [7]:
df

,annee,code_sexe,sexe,code_secteur_cnu,secteur_disciplinaire,code_groupe_corps,groupe_de_corps,code_uai,etablissement,code_academie,academie,code_region,region,code_pres,pres,code_idex,idex,beneficiaires,geo_localisation
0,1993,1,Hommes,1,Droit et sciences économiques,1,Professeurs et assimilés,0341087X,Université Montpellier 1,11,Montpellier,91,Languedoc-Roussillon,0342255S,PRES Université Montpellier Sud de France,0,Hors IDEX,2,"43.61493,3.873641"
1,1993,1,Hommes,1,Droit et sciences économiques,1,Professeurs et assimilés,0370800U,Université de Tours - François Rabelais,18,Orléans-Tours,24,Centre,0371692N,PRES Centre-Val de Loire Université,0,Hors IDEX,1,"47.38366,0.675319"
2,1993,1,Hommes,1,Droit et sciences économiques,1,Professeurs et assimilés,0673021V,Université de Strasbourg,15,Strasbourg,42,Alsace,0000000X,Hors PRES,8,UNISTRA,2,"48.58004,7.765612"
3,1993,1,Hommes,1,Droit et sciences économiques,1,Professeurs et assimilés,0691775E,Université Lyon 2 - Lumière,10,Lyon,82,Rhône-Alpes,0694094A,PRES Université de Lyon,0,Hors IDEX,2,"45.74955,4.837425"
4,1993,1,Hommes,1,Droit et sciences économiques,1,Professeurs et assimilés,0751717J,Université Paris 1 - Panthéon Sorbonne,1,Paris,11,Île-de-France,0755581J,PRES Hautes Etudes - Sorbonne - Arts et Métiers,0,Hors IDEX,9,"48.8468,2.34486"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11469,2010,2,Femmes,3,Sciences,2,Maîtres de conférences et assimilés,0333232J,Bordeaux INP,4,Bordeaux,72,Aquitaine,0333178A,PRES Université de Bordeaux,2,IDEX Bordeaux,2,"44.80613,-0.60754"
11470,2010,2,Femmes,3,Sciences,2,Maîtres de conférences et assimilés,0341087X,Université Montpellier 1,11,Montpellier,91,Languedoc-Roussillon,0342255S,PRES Université Montpellier Sud de France,0,Hors IDEX,1,"43.61493,3.873641"
11471,2010,2,Femmes,3,Sciences,2,Maîtres de conférences et assimilés,0350936C,Université Rennes 1,14,Rennes,53,Bretagne,0352756F,PRES Université européenne de Bretagne,0,Hors IDEX,5,"48.11605,-1.67414"
11472,2010,2,Femmes,3,Sciences,2,Maîtres de conférences et assimilés,0542493S,Université de Lorraine,12,Nancy-Metz,41,Lorraine,0542471T,PRES Université de Lorraine,0,Hors IDEX,9,"48.69623,6.176543"


In [12]:
#add data to secteur table
sql = "INSERT INTO secteur (code_secteur_cnu,secteur_disciplinaire ) VALUES (%s,%s)"
x= df["code_secteur_cnu"].unique().astype(int)
y= df["secteur_disciplinaire"].unique()
val = [(x[0].item(),y[0]),
       (x[1].item(),y[1]),
       (x[2].item(),y[2]),
       (x[3].item(),y[3]),
       (x[4].item(),y[4])]
mycursor.executemany(sql, val)

In [19]:
# add data to type bourse
sql = "INSERT INTO type_bourse (code_groupe_corps,groupe_de_corps ) VALUES (%s,%s)"
x= df["code_groupe_corps"].unique().astype(int)
y= df["groupe_de_corps"].unique()
val = [(x[0].item(),y[0]),
       (x[1].item(),y[1]),
       (x[2].item(),y[2])
      ]
mycursor.executemany(sql, val)

In [20]:
# add data to type excellence
sql = "INSERT INTO type_excellence (code_idex,code_pres,idex,pres ) VALUES (%s,%s,%s,%s)"
dx = df.groupby(['code_idex', 'code_pres','idex','pres']).size().reset_index().rename(columns={0:'count'})
#x= df["code_groupe_corps"].unique().astype(int)
val = []
for index, row in dx.iterrows():
        val.append((row["code_idex"],row["code_pres"],row["idex"],row["pres"]))
#print(val)
mycursor.executemany(sql, val)

In [21]:
# add data to region
sql = "INSERT INTO region (code_region,region ) VALUES (%s,%s)"
dx = df.groupby(['code_region', 'region']).size().reset_index().rename(columns={0:'count'})
val = []
for index, row in dx.iterrows():
        val.append((row["code_region"],row["region"]))
#print(val)
mycursor.executemany(sql, val)

In [22]:
# add data to academie
sql = "INSERT INTO Academie (code_academie,academie,code_region ) VALUES (%s,%s,%s)"
dx = df.groupby(['code_academie', 'academie','code_region']).size().reset_index().rename(columns={0:'count'})
val = []
for index, row in dx.iterrows():
        val.append((row["code_academie"],row["academie"],row["code_region"]))
#print(val)
mycursor.executemany(sql, val)

In [23]:
# add data to institut
sql = "INSERT INTO institut (code_uai,etablissement,geo_localisation,code_academie ) VALUES (%s,%s,%s,%s)"
dx = df.groupby(['code_uai', 'etablissement','geo_localisation','code_academie']).size().reset_index().rename(columns={0:'count'})
val = []
for index, row in dx.iterrows():
        val.append((row["code_uai"],row["etablissement"],row["geo_localisation"],row['code_academie']))
#print(val)
mycursor.executemany(sql, val)

In [35]:
# add data to Bourse
sql = "INSERT INTO Bourse (id_bourse,code_uai,code_secteur_cnu,code_groupe_corps,code_idex,code_pres) VALUES (%s,%s,%s,%s,%s,%s)"
dx = df.groupby(['code_uai', 'code_secteur_cnu','code_groupe_corps','code_idex','code_pres']).size().reset_index().rename(columns={0:'count'})
dx["id_bourse"]= dx.index+311
val = []
for index, row in dx.iterrows():
        val.append((index+311,row["code_uai"],row["code_secteur_cnu"],row["code_groupe_corps"],row['code_idex'],row['code_pres']))
#print(val)
mycursor.executemany(sql, val)

In [36]:
#add id_bourse to the dataframe 
df2 = df.merge(dx, how='left', on=('code_uai', 'code_secteur_cnu','code_groupe_corps','code_idex','code_pres'))

# add data to groupe_candidats
sql = "INSERT INTO groupe_candidats (id_groupe, sexe, annee, beneficiaires,id_bourse) VALUES (%s,%s,%s,%s,%s)"
dy = df2.groupby(['sexe','annee','beneficiaires','id_bourse']).size().reset_index().rename(columns={0:'count'})
val = []
for index, row in dy.iterrows():
        val.append((index+15,row["sexe"],row["annee"],row["beneficiaires"],row['id_bourse']))

mycursor.executemany(sql, val)

In [37]:
mydb.commit()

In [38]:
mycursor.close()
mydb.close()